In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-score-classification/train.csv
/kaggle/input/credit-score-classification/test.csv


In [ ]:
# Remove Warning
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# Importing dataset
credit_df = pd.read_csv('/kaggle/input/credit-score-classification/train.csv')

In [ ]:
credit_df.shape

In [ ]:
credit_df.shape

In [ ]:
credit_df.head(10)

### 3. Exploratory Data Analysis (EDA)

3.1 Datatypes, Missing Data,Finding Categorical Col, and Summary Statistics

In [ ]:
credit_df.info()

In [ ]:
# Viewing all type of data we have
credit_df.dtypes.value_counts()

In [ ]:
# find categorical Columns and put that into a 

categorical = [var for var in credit_df.columns if credit_df[var].dtype=='O']

print('There are {} categorical variables\n'.format(len(categorical)))

print('The categorical variables are :\n\n', categorical)

In [ ]:
credit_df['Credit_Score'].value_counts()


In [ ]:
credit_df[categorical].head()

In [ ]:
categorical

In [ ]:
credit_df['Month'].value_counts()


In [ ]:
credit_df['Month'].nunique()


In [ ]:
for col in credit_df:
    print(f"Value counts for '{col}':")
    print(credit_df[col].value_counts())
    print("\n" + "-"*50 + "\n")


In [ ]:
for col in credit_df:
    print(f"Value counts for '{col}':")
    print(credit_df[col].nunique())
    print("\n" + "-"*50 + "\n")

In [ ]:
credit_df['Type_of_Loan'].value_counts()

In [ ]:
### Changing the datatype of the above mentioned columns to category

credit_df.Month = credit_df.Month.astype('category')
credit_df.Occupation = credit_df.Occupation.astype('category')
credit_df.Type_of_Loan = credit_df.Type_of_Loan.astype('category')
credit_df.Credit_Mix = credit_df.Credit_Mix.astype('category')
credit_df.Payment_of_Min_Amount = credit_df.Payment_of_Min_Amount.astype('category')
credit_df.Payment_Behaviour = credit_df.Payment_Behaviour.astype('category')
credit_df.Credit_Score = credit_df.Credit_Score.astype('category')

In [ ]:
credit_df.info()

In [ ]:
credit_df['Age'].astype(str).str.contains('_').any()

In [ ]:
credit_df[credit_df['Age'].astype(str).str.contains('_')]

The Columns type we need to Change: `Age`,`Annual_Income`,`Num_of_Loan`,`Num_of_Delayed_Payment `,`Changed_Credit_Limit`,
`Outstanding_Debt`,`Amount_invested_monthly`,`Monthly_Balance`

In [ ]:
# Function to remove leading/trailing underscores and return cleaned value
def removeUnderscore(value):
    first_index = 0
    last_index = len(value) - 1

    while first_index <= last_index:
        if value[first_index] == '_':
            first_index += 1
        if value[last_index] == '_':
            last_index -= 1

        # Break when there are no underscores in the substring
        if '_' not in value[first_index : last_index + 1]:
            cleaned_value = value[first_index : last_index + 1]
            return 0 if cleaned_value == '' else cleaned_value


# Function to clean and convert object-type columns to float
def un_clean(columns):
    for column in columns:
        # Convert all values to strings
        raw_data = [str(value) for value in list(credit_df[column])]
        cleaned_data = []

        for value in raw_data:
            if value.lower() == 'nan':
                cleaned_data.append(float('nan'))
            else:
                cleaned_value = removeUnderscore(value)
                cleaned_data.append(float(cleaned_value))

        credit_df[column] = cleaned_data


# Apply to selected columns
un_clean([
    'Age', 
    'Annual_Income', 
    'Num_of_Loan', 
    'Num_of_Delayed_Payment', 
    'Outstanding_Debt', 
    'Changed_Credit_Limit',
    'Amount_invested_monthly', 
    'Monthly_Balance'
])


In [ ]:
credit_df.info()

The Columns type we need to Change: `Age`,`Annual_Income`,`Num_of_Loan`,`Num_of_Delayed_Payment `,`Changed_Credit_Limit`,
`Outstanding_Debt`,`Amount_invested_monthly`,`Monthly_Balance` Now we can see that all this column is changes object to float

Now We can see there are missing values we have in some of the columns that we need to remove

In [ ]:
# Missing Values
credit_df.isnull().sum()

In [ ]:
### Missing data by columns in the dataset Order Wise

credit_df.isnull().sum().sort_values(ascending = False)

In [ ]:
missing = credit_df.isnull().sum()
missing_percent = (missing / len(credit_df)) * 100
missing_table = pd.DataFrame({'Missing Values': missing, 'Percent (%)': missing_percent})
missing_table = missing_table[missing_table['Missing Values'] > 0].sort_values(by='Percent (%)', ascending=False)
print(missing_table)


From the above data, we observe that the columns `Monthly_Inhand_Salary`, `Type_of_Loan`, `Name`, `Credit_History_Age`, `Num_of_Delayed_Payment`, `Amount_invested_monthly`, `Num_Credit_Inquiries`, and `Monthly_Balance` contain missing values. Next, we will analyze the distribution of these missing values to determine whether they are clustered in specific sections or scattered throughout the dataset.

Only Missing values Bar Chart repreesentation

In [ ]:
import plotly.express as px

missing_df = credit_df.isnull().sum().reset_index()
missing_df.columns = ['column', 'missing_count']
missing_df = missing_df[missing_df['missing_count'] > 0]

fig = px.bar(missing_df, x='column', y='missing_count',
             title='Missing Values per Column',
             color='missing_count',
             color_continuous_scale='sunset')
fig.show()


In [ ]:
# Importing seaborn & matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
### Visual representation of the missing data in the dataset

plt.figure(figsize=(12, 6))
sns.heatmap(credit_df.isnull(), cbar=False, cmap="tab10")
plt.title("Heatmap of Missing Values")
plt.show()


In [ ]:
# Compute the null correlation matrix
null_corr = credit_df.isnull().corr()

# Plot with a diverging color palette
plt.figure(figsize=(20, 12))
sns.heatmap(null_corr, cmap='coolwarm', annot=True, linewidths=0.5, linecolor='gray')
plt.title("Correlation of Missing Values", fontsize=14)
plt.show()


**From the above data, we can see that the missing values in the data are scattered, not clustered at a single place.**

In [ ]:
### Summary statistics of the numerical columns in the dataset

credit_df.describe()

From the above summary statistics, we can see that there are `outliers` present in the data. We will take care of these in the next sections(Preprocessing).



3.2 Feature Analysis

3.2.1 Categorical variable - Occupation

In [ ]:
### Value counts of the column - Occupation

occupation_count = credit_df['Occupation'].value_counts(dropna = False)
occupation_count

the parameter `dropna=False` tells pandas to include missing values (NaN) in the count.

🔍 Explanation:
By default, value_counts() ignores NaN values.

If you want to see how many missing values are present in the column along with other values, use dropna=False.

In [ ]:
### Bar graph showing the value counts of the column - Occupation

sns.set(rc = {'figure.figsize': (20, 10)})
sns.barplot(x=occupation_count.index, y=occupation_count.values)
plt.title('Bar graph showing the value counts of the column - Occupation')
plt.ylabel('Count', fontsize = 12)
plt.xlabel('Occupation', fontsize = 12)

In [ ]:
credit_df['Occupation'].nunique()

From the above graph, we can see that most of the jobs are 'unnamed'.



In [ ]:
occupation_count.values

In [ ]:
occupation_count.index

In [ ]:

# Set the display option to show all columns
pd.set_option('display.max_columns', None)

# Now, filter and display your DataFrame
good_credit_df = credit_df[credit_df['Credit_Score'] == 'Good']

# This will now print all the columns without skipping any
good_credit_df

In [ ]:
### Distribution of Credit_Score for each Occupation

sns.catplot(x='Credit_Score', 
            col='Occupation', 
            data=credit_df, 
            kind='count', 
            col_wrap=4, 
            height=4, 
            aspect=1.2)

plt.show()


By observing we can see that all have the `Standard` Credit_Score

3.2.2 Categorical variable - Type of Loan

In [ ]:
index_values = ~credit_df['Type_of_Loan'].isnull().values
loan_type_data = list(credit_df['Type_of_Loan'][index_values])
# loan_type_data

Goal of Below Code:Count how many times each type of loan appears across all customers.

In [ ]:
### Create a dictionary to store the counts of all the various loan types

loan_type_dict = dict()
for value in loan_type_data:
    values = value.split(',')
    for each_value in values:
        loan_type = each_value.strip(' ')
        if 'and' in loan_type:
            loan_type = loan_type[4 : ]
        if loan_type in loan_type_dict:
            loan_type_dict[loan_type] += 1 # This is for Exsisting one Counting
        else:
            loan_type_dict[loan_type] = 1  # This is for new one 

loan_type_dict

In [ ]:
### Bar graph showing the counts of the column - Type_of_Loan

sns.set(rc = {'figure.figsize': (15, 10)})
sns.barplot(x=list(loan_type_dict.keys()), y=list(loan_type_dict.values()))
plt.title('Bar graph showing the counts of the column - Type_of_Loan')
plt.ylabel('Count', fontsize = 12)
plt.xlabel('Type_of_Loan', fontsize = 12)

From the above graph, we can see that the `Payday Loans` and `Credit-Builder Loans` are the highest occurrences of loans among all the other loans.



3.2.3 Categorical variable - Credit_MIx¶

In [ ]:
### Value counts of the column - Credit_Mix

credit_mix_count = credit_df['Credit_Mix'].value_counts(dropna = False)
credit_mix_count

In [ ]:
### Bar graph showing the value counts of the column - Credit_Mix

sns.set(rc = {'figure.figsize': (6, 6)})
sns.barplot(x=credit_mix_count.index, y=credit_mix_count.values, alpha = 0.8)
plt.title('Bar graph showing the value counts of the column - Credit_Mix')
plt.ylabel('Number of Occurrences', fontsize = 12)
plt.xlabel('Credit Mix', fontsize = 12)
plt.show()

From the above graph, we can see that most of the customers have a `Standard` credit mix.



In [ ]:
### Distribution of Credit_Score for each Credit_Mix

sns.catplot(
    x='Credit_Score',
    col='Credit_Mix',
    data=credit_df,
    kind='count',
    col_wrap=3,
    height=4,        # size of each small plot
    aspect=1         # width-to-height ratio
)

plt.show()


“From the above graphs, we can observe a strong relationship between `Credit_Mix` and `Credit_Score`. For example, when Credit_Mix is `Bad`, the majority of Credit_Score values are `Poor`. Similarly, a `Good` Credit_Mix is often associated with a `Good` Credit_Score.”

3.2.4 Categorical variable - Payment_of_Min_Amount

This is all the Category Columns We have

In [ ]:
# find categorical Columns and put that into a 
# Find all columns with 'category' dtype
cat_cols = [var for var in credit_df.columns if credit_df[var].dtype.name == 'category']

print('There are {} category variables\n'.format(len(cat_cols)))
print('The category variables are:\n\n', cat_cols)


In [ ]:
### Value counts of the column - Payment_of_Min_Amount

min_amount_count = credit_df['Payment_of_Min_Amount'].value_counts(dropna = False)
min_amount_count

In [ ]:
### Bar graph showing the value counts of the column - Payment_of_Min_Amount

sns.set(rc = {'figure.figsize': (5, 5)})
sns.barplot(x=min_amount_count.index, y=min_amount_count.values, alpha = 0.8)
plt.title('Bar graph showing the value counts of the column - Payment_of_Min_Amount')
plt.ylabel('Number of Occurrences', fontsize = 12)
plt.xlabel('Payment of Minimum Amount', fontsize = 12)
plt.show()

From the above graph, we can see that most of the customer's paid a minimum amount for their loans.

In [ ]:
### Distribution of Payment_of_Min_Amount for each Credit Score

sns.catplot(
    x='Payment_of_Min_Amount', 
    col='Credit_Score', 
    data=credit_df, 
    kind='count', 
    col_wrap=3,
    height=4,
    aspect=1
)

plt.show()

**Observations:**
When `Payment_of_Min_Amount` is "No" → `Credit_Score` is mostly "Good".

Suggests that people who don’t just pay the minimum amount might be financially stronger, thus having better credit scores.

When `Credit_Score` is "Poor" → `Payment_of_Min_Amount` is mostly "Yes".

Indicates that struggling borrowers are more likely to pay only the minimum required, which could hurt their credit score.

When `Credit_Score` is "Standard" → `Payment_of_Min_Amount` is also mostly "Yes".

Shows that paying only the minimum doesn’t necessarily mean "Poor" score, but it tends to keep people in a middle ("Standard") range instead of "Good".

From the above graph we can Observe:
Paying only the minimum amount is generally associated with lower or average credit scores.

Not paying the minimum amount (i.e., paying more) is correlated with a Good credit score.

This could indicate that payment habits are an important predictor for creditworthiness — people paying more than the minimum may have better financial discipline or income, leading to stronger credit profiles.

In [ ]:
credit_df['Payment_of_Min_Amount'].count()

3.2.5 Numerical variable - Age

In [ ]:
### Understanding the distribution of the column - Age
# This is for old Version because now distplot become displot
sns.distplot(credit_df['Age'], label = 'Skewness: %.2f'%(credit_df['Age'].skew()))
plt.legend(loc = 'best')
plt.title('Customer Age Distribution')

In [ ]:
# This is for new Version
sns.displot(credit_df['Age'],kind="kde", label = 'Skewness: %.2f'%(credit_df['Age'].skew()))
plt.legend(loc='best')
plt.title('Customer Age Distribution')

In [ ]:
credit_df['Age'].value_counts()

In [ ]:
credit_df['Age'].count()

From the above graph, we can see that the above graph has a high degree of skewness.
A skewness of 9.21 is extremely high, which means your data is very heavily positively skewed (right-skewed).

3.2.6 Numerical variable - Monthly_Inhand_Salary

In [ ]:
### Understanding the distribution of the column - Monthly_Inhand_Salary

sns.distplot(credit_df['Monthly_Inhand_Salary'], label = 'Skewness: %.2f'%(credit_df['Monthly_Inhand_Salary'].skew()))
plt.legend(loc = 'best')
plt.title('Customer Monthly Salary Distribution')

In [ ]:
sns.displot(
    credit_df['Monthly_Inhand_Salary'], 
    kde=True, 
    stat="density",  # Show density instead of counts
    label='Skewness: %.2f' % (credit_df['Monthly_Inhand_Salary'].skew())
)

plt.legend(loc='best')
plt.title('Customer Monthly Salary Distribution')
plt.show()


From the above graph, we can see that the distribution is right skewed and has a slight degree of skewness.

In [ ]:
### Monthly Inhand Salary distribution by Credit Score

grid = sns.FacetGrid(credit_df, col = 'Credit_Score')
grid.map(sns.distplot, 'Monthly_Inhand_Salary')

In [ ]:
### Merging the above graphs into one

plt.figure(figsize=(8, 5))

for score in ['Good', 'Poor', 'Standard']:
    sns.kdeplot(
        data=credit_df[credit_df['Credit_Score'] == score]['Monthly_Inhand_Salary'],
        label=f'Credit Score = {score}'
    )

plt.xlabel('Monthly Inhand Salary')
plt.ylabel('Density')
plt.title('Customer Monthly Inhand Salary by Credit Score')
plt.legend()
plt.show()


From the above graph, we can see that most of the customer's who have a Poor credit score have a low monthly inhand salary than compared to the customer's who have a Standard and a Good credit score.

3.2.7 Numerical variable - Interest_Rate

In [ ]:
### Understanding the distribution of the column - Interest_Rate

sns.distplot(credit_df['Interest_Rate'], label = 'Skewness: %.2f'%(credit_df['Interest_Rate'].skew()))
plt.legend(loc = 'best')
plt.title('Customers Interest Rate Distribution')

From the above graph, we can see that the above graph has a high degree of skewness.
So,
dataset’s skewness = 9.01, that’s extremely high — meaning our data is very heavily right-skewed (positively skewed).

Here’s what it implies:

Shape → The distribution has a long tail on the right side.

Mean > Median → The mean will be much larger than the median.

Outliers → You likely have extreme high-value outliers pulling the tail to the right.

Normality → The data is far from normal; statistical models that assume normality (e.g., many parametric tests) may perform poorly unless transformed.

Handling → Often, such data benefits from transformations like log, square root, or Box-Cox to reduce skewness before modeling.

📌 `Rule of thumb for skewness:`

`0` → perfectly symmetric

`0 to ±0.5` → fairly symmetric

`±0.5 to ±1` → moderately skewed

`±1` → highly skewed

`9.01` → extremely skewed 🚨(It's Our)



3.2.8 Numerical variable - Outstanding_Debt

In [ ]:
### Understanding the distribution of the column - Outstanding_Debt

sns.distplot(credit_df['Outstanding_Debt'], label = 'Skewness: %.2f'%(credit_df['Outstanding_Debt'].skew()))
plt.legend(loc = 'best')
plt.title("Customer's Outstanding Debt  Distribution")

In [ ]:
credit_scores = ['Good', 'Poor', 'Standard']

for score in credit_scores:
    count = credit_df[
        (credit_df['Credit_Score'] == score) &
        (credit_df['Outstanding_Debt'].notnull())
    ].shape[0]
    print(f"Credit Score = {score} → Count = {count}")



In [ ]:
### Outstanding Debt distribution by Credit Score

grid = sns.FacetGrid(credit_df, col = 'Credit_Score')
grid.map(sns.distplot, 'Outstanding_Debt')

In [ ]:
plt.figure(figsize=(8, 5))

for score in ['Good', 'Poor', 'Standard']:
    data = credit_df.loc[credit_df['Credit_Score'] == score, 'Outstanding_Debt']
    sns.kdeplot(data, label=f'Credit Score = {score}')

plt.xlabel("Customer's Outstanding Debt")
plt.ylabel("Density")
plt.title("Customer's Outstanding Debt by Credit Score")
plt.legend()
plt.show()


From the above graph, we can see that customer's who have a Good credit score have very low outstanding debt than compared to the customer's who have Standard and Poor credit score.

4. Data preprocessing

Data preprocessing is the process of getting our dataset ready for model training. In this section, we will perform the following preprocessing steps:

1. Detect and remove outliers in numerical variables
2. Drop and fill missing values
3. Feature engineering
4. Data trasformation
5. Feature encoding

4.1 Detect and remove outliers in numerical variables

Outliers are data points that have extreme values and they do not conform with the majority of the data. It is important to address this because outliers tend to skew our data towards extremes and can cause inaccurate model predictions. I will use the Tukey method to remove these outliers.

Here, we will write a function that will loop through a list of features and detect outliers in each one of those features. In each loop, a data point is deemed an outlier if it is less than the first quartile minus the outlier step or exceeds third quartile plus the outlier step. The outlier step is defined as 1.5 times the interquartile range. Once the outliers have been determined for one feature, their indices will be stored in a list before proceeding to the next feature and the process repeats until the very last feature is completed. Finally, using the list with outlier indices, we will count the frequencies of the index numbers and return them if their frequency exceeds n times.

In [ ]:
from collections import Counter

In [ ]:
def detect_outliers(df, n, features_list):
    outlier_indices = [] 
    for feature in features_list: 
        Q1 = np.percentile(df[feature], 25)
        Q3 = np.percentile(df[feature], 75)
        IQR = Q3 - Q1
        outlier_step = 1.5 * IQR 
        outlier_list_col = df[(df[feature] < Q1 - outlier_step) | (df[feature] > Q3 + outlier_step)].index
        outlier_indices.extend(outlier_list_col) 
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(key for key, value in outlier_indices.items() if value > n) 
    return multiple_outliers

In [ ]:
numerical_columns = list(credit_df.select_dtypes('number').columns)
print('Numerical columns: {}'.format(numerical_columns))
outliers_to_drop = detect_outliers(credit_df, 2, numerical_columns)
print("We will drop these {} indices: ".format(len(outliers_to_drop)), outliers_to_drop)

Now let's look at the data present in the rows.

In [ ]:
credit_df.iloc[outliers_to_drop, :]

Above all the thing is outlier so we are going to drop it.

We will drop these rows from the dataset.

In [ ]:
### Drop outliers and reset index

print("Before: {} rows".format(len(credit_df)))
credit_df = credit_df.drop(outliers_to_drop, axis = 0).reset_index(drop = True)
print("After: {} rows".format(len(credit_df)))

In [ ]:
### Lets look at the new dataset

credit_df

4.2 Drop and fill missing values

Here, we will drop the columns - ID, Customer_ID, Name, SSN, Credit_Mix, Num_of_Loan, Credit_Utilization_Ratio, Credit_History_Age, Payment_Behavior, Annual_Income, Monthly_Balance, Num_Bank_Accounts, Num_Credit_Card from the datasets.

In [ ]:
### Dropping the columns from the dataset

credit_df.drop(['ID', 'Customer_ID', 'Name', 'SSN', 'Credit_Mix', 'Num_of_Loan',
             'Credit_Utilization_Ratio', 'Credit_History_Age', 'Payment_Behaviour', 
             'Annual_Income', 'Monthly_Balance', 'Num_Bank_Accounts', 'Num_Credit_Card'], axis = 1, inplace = True)
credit_df

In [ ]:
credit_df.info()

In [ ]:
### Looking at the missing values in the dataset

credit_df.isnull().sum().sort_values(ascending = False)

From the above data, we can see that there are missing values in the columns - `Monthly_Inhand_Salary`, `Type_of_Loan`, `Num_of_Delayed_Payment`, `Amount_invested_monthly`, `Num_Credit_Inquiries`. Here, we will focus on removing the missing values in the columns - `Monthly_Inhand_Salary`, `Num_of_Delayed_Payment`, `Amount_invested_monthly`, and `Num_Credit_Inquiries`. However, we will replace the missing values in the column - Type_of_Loan in the Feature Engineering section.

4.2.1 Handling missing values - Monthly_Inhand_Salary

Here, for replacing the missing values in the column - Monthly_Inhand_Salary, we will use the column Credit_Score and find the mean of the salary based on the Credit Score.

In [ ]:
# Finding mean Monthly_Inhand_Salary for each Credit_Score category
salary_means = credit_df.groupby('Credit_Score')['Monthly_Inhand_Salary'].mean()

salary_good_mean = salary_means.get('Good')
salary_poor_mean = salary_means.get('Poor')
salary_standard_mean = salary_means.get('Standard')

(salary_good_mean, salary_poor_mean, salary_standard_mean)


In [ ]:
### Finding the indices of the rows where Monthly_Inhand_Salary is null

index_values = list(credit_df['Monthly_Inhand_Salary'].isnull())
index_values

In [ ]:
### Replacing the missing values in the column Monthly_Inhand_Salary using the decision logic

for index in range(len(credit_df)):
    if index_values[index]:
        if credit_df['Credit_Score'][index] == 'Good':
            credit_df['Monthly_Inhand_Salary'][index] = salary_good_mean
        elif credit_df['Credit_Score'][index] == 'Poor':
            credit_df['Monthly_Inhand_Salary'][index] = salary_poor_mean
        else:
            credit_df['Monthly_Inhand_Salary'][index] = salary_standard_mean

In [ ]:
### Checking if there are any missing values of Monthly_Inhand_Salary in the dataset

credit_df['Monthly_Inhand_Salary'].isnull().sum()

We replaced all the missing values present in the column - `Monthly_Inhand_Salary`.

4.2.2 Handling missing values - Num_of_Delayed_Payment

Here, we will use the median to replace the missing values in the column - `Num_of_Delayed_Payment`.

In [ ]:
credit_df['Num_of_Delayed_Payment'].isnull().sum()

In [ ]:
### Finding the median value of the column - Num_of_Delayed_Payment in the dataset

#payment_index = list(~train_df['Num_of_Delayed_Payment'].isnull())
#median_payment = np.median(train_df['Num_of_Delayed_Payment'].loc[payment_index])
median_payment = credit_df['Num_of_Delayed_Payment'].median()  #Pandas’ built-in .median() already ignores NaNs by default

median_payment

In [ ]:
### Replacing the missing values of the column - Num_of_Delayed_Payment in the dataset

credit_df['Num_of_Delayed_Payment'].fillna(median_payment, inplace = True)

In [ ]:
credit_df['Num_of_Delayed_Payment'].isnull().sum()

We replaced all the missing values present in the column - `Num_of_Delayed_Payment`.

4.2.3 Handling missing values - `Amount_invested_monthly`

In [ ]:
credit_df['Amount_invested_monthly'].isnull().sum()

Here, we will use the median to replace the missing values in the column - Amount_invested_monthly.

In [ ]:
### Finding the median value of the column - Amount_invested_monthly in the dataset

median_amount = credit_df['Amount_invested_monthly'].median()
median_amount

In [ ]:
### Replacing the missing values of the column - Amount_invested_monthly in the dataset

credit_df['Amount_invested_monthly'].fillna(median_amount, inplace = True)

In [ ]:
credit_df['Amount_invested_monthly'].isnull().sum()

We replaced all the missing values present in the column - Amount_invested_monthly.

4.2.4 Handling missing values - Num_Credit_Inquiries

Here, we will use the median to replace the missing values in the column - `Num_Credit_Inquiries`.

In [ ]:
### Finding the median value of the column - Num_Credit_Inquiries in the dataset
median_inquiries = credit_df['Num_Credit_Inquiries'].median()
median_inquiries

In [ ]:
### Replacing the missing values of the column - Num_Credit_Inquiries in the dataset

credit_df['Num_Credit_Inquiries'].fillna(median_inquiries, inplace = True)

In [ ]:
### Checking if there are any missing values of Num_Credit_Inquiries in the dataset
credit_df['Num_Credit_Inquiries'].isnull().sum()

We replaced all the missing values present in the column - `Num_Credit_Inquiries`.

In [ ]:
### Looking if the dataset has any more missing values apart from Type_of_Loan

credit_df.isnull().sum().sort_values(ascending = False)

In [ ]:
credit_df.info()

Since, there are no missing values in the data apart from the data in the column - `Type_of_Loan` (which we will deal in the `Feature Engineering`), we can proceed to perform Feature Engineering.

4.3 Feature Engineering

Feature engineering is arguably the most important art in machine learning. It is the process of creating new features from existing features to better represent the underlying problem to the predictive models resulting in improved model accuracy on unseen data.


Here, we focus on creating new columns for:

1.Individual columns for `Type_of_Loan`

4.3.1 Individual columns for `Type_of_Loan`

Here, we will create 8 different columns using the loan_type_dict dictionary. Here, we will not consider the value Not Specified for the loan type.

In [ ]:
### Looking at the loan_type_dict

loan_type_dict

In [ ]:
# This is for how many unique types of loan we have
unique_loans = set()  # to store unique loan types

for entry in loan_type_data:
    # Replace ' and ' with ', ' so we can split everything by comma
    clean_entry = entry.replace(' and ', ', ')
    
    # Split by comma
    for loan in clean_entry.split(','):
        loan = loan.strip()  # remove extra spaces
        if loan and loan != "Not Specified":  # skip empty & unwanted
            unique_loans.add(loan)

print(unique_loans)


In [ ]:
### Creating 8 different lists for each loan type
# Here basically we are createing a list with the same length of the dataset and then felling all the values with the zero.
auto_loan = [0] * (len(credit_df))
credit_builder_loan = [0] * (len(credit_df))
personal_loan = [0] * (len(credit_df))
home_equity_loan = [0] * (len(credit_df))
mortgage_loan = [0] * (len(credit_df))
student_loan = [0] * (len(credit_df))
debt_consolidation_loan = [0] * (len(credit_df))
payday_loan = [0] * (len(credit_df))

In [ ]:
pd.Series(loan_type_data).nunique()

In [ ]:
len(loan_type_data)

In [ ]:
for index in range(len(loan_type_data)):
    ### For Auto Loan
    if 'Auto' in loan_type_data[index]:
        auto_loan[index] = 1
    
    ### For Credit Builder Loan
    if 'Credit-Builder' in loan_type_data[index]:
        credit_builder_loan[index] = 1
        
    ### For Personal Loan
    if 'Personal' in loan_type_data[index]:
        personal_loan[index] = 1
    
    ### For Home Equity Loan
    if 'Home' in loan_type_data[index]:
        home_equity_loan[index] = 1
    
    ### For Mortgage Loan
    if 'Mortgage' in loan_type_data[index]:
        mortgage_loan[index] = 1
    
    ### For Student Loan
    if 'Student' in loan_type_data[index]:
        student_loan[index] = 1
        
    ### For Debt Consolidation loan
    if 'Debt' in loan_type_data[index]:
        debt_consolidation_loan[index] = 1
    
    ### For Payday loan
    if 'Payday' in loan_type_data[index]:
        payday_loan[index] = 1


In [ ]:
### Adding the new columns to the dataset

credit_df['Auto_Loan'] = auto_loan
credit_df['Credit_Builder_Loan'] = credit_builder_loan
credit_df['Personal_Loan'] = personal_loan
credit_df['Home_Enquity_Loan'] = home_equity_loan
credit_df['Mortgage_Loan'] = mortgage_loan
credit_df['Student_Loan'] = student_loan
credit_df['Debt_Consolidation_Loan'] = debt_consolidation_loan
credit_df['Payday_Loan'] = payday_loan

In [ ]:
### Removing the column - Type_of_loan

credit_df.drop(['Type_of_Loan'], axis = 1, inplace = True)

In [ ]:
credit_df.isnull().sum()

Now we can see that we don't have any types of null values now 

In [ ]:
# Now this is how our dataset looks like
credit_df

In [ ]:
credit_df.info()

In [ ]:
# Set the display option to show all columns
pd.set_option('display.max_columns', None)

# Now, filter and display your DataFrame
good_credit_df = credit_df[credit_df['Credit_Score'] == 'Good']

# This will now print all the columns without skipping any
good_credit_df

In [ ]:
# Set the display option to show all columns
pd.set_option('display.max_columns', None)

# Now, filter and display your DataFrame
Poor_credit_df = credit_df[credit_df['Credit_Score'] == 'Poor']

# This will now print all the columns without skipping any
Poor_credit_df

In [ ]:
# Set the display option to show all columns
pd.set_option('display.max_columns', None)

# Now, filter and display your DataFrame
Standard_credit_df = credit_df[credit_df['Credit_Score'] == 'Standard']

# This will now print all the columns without skipping any
Standard_credit_df

4.4 Data Transformation

From the dataset, we can see that there is a lot of skewness in the numerical columns of the dataset. Here, we will focus on removing the skewness in the data using `log transformation`.

4.4.1 Log Transforming the column - `Age`

In [ ]:
credit_df['Age'].skew()

In [ ]:
### Understanding the distribution of the column - Age

sns.distplot(credit_df['Age'], 
            label = 'Skewness: %.2f'%(credit_df['Age'].skew())
           )
plt.legend(loc = 'best')
plt.title('Customer Age Distribution')

In [ ]:
# This is tell us in our dataset we have the negative values or not
(credit_df['Age'] < 0).any()


In [ ]:
# It's tell us how many negative values we have in our dataset
(credit_df['Age'] < 0).sum()


Use Yeo-Johnson transformation — a more flexible method that can handle zero and negative values without shifting:
Here we use the Yeo-Johnson Methode because we have the negative values to solve this thing we use this methode

In [ ]:
### Understanding the distribution of the data log(Age)
# Applying log Transformation
# modified_age = [np.log(age) if age > 0 else 0 for age in train_df['Age']]
# train_df['Age'] = modified_age
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer(method='yeo-johnson')
credit_df['Age'] = pt.fit_transform(credit_df[['Age']])

sns.distplot(credit_df['Age'], label = 'Skewness: %.2f'%(credit_df['Age'].skew()))
plt.legend(loc = 'best')
plt.title('Customer Age Distribution')

From the above graph, we can see that the degree of skewness is significantly reduced than compared to the skewness in the original data.

4.4.2 Log Transforming the column - Monthly_Inhand_Salary

In [ ]:
### Understanding the distribution of the column - Monthly_Inhand_Salary

sns.distplot(credit_df['Monthly_Inhand_Salary'], label = 'Skewness: %.2f'%(credit_df['Monthly_Inhand_Salary'].skew()))
plt.legend(loc = 'best')
plt.title('Customer Monthly Inhand Salary Distribution')


In [ ]:
# This is tell us in our dataset we have the negative values or not
(credit_df['Monthly_Inhand_Salary'] < 0).any()

In [ ]:
# It's tell us how many negative values we have in our dataset
(credit_df['Monthly_Inhand_Salary'] < 0).sum()


In [ ]:
### Understanding the distribution of the data log(Monthly_Inhand_Salary)

modified_salary = [np.log(salary) if salary > 0 else 0 for salary in credit_df['Monthly_Inhand_Salary']]
credit_df['Monthly_Inhand_Salary'] = modified_salary

sns.distplot(credit_df['Monthly_Inhand_Salary'], label = 'Skewness: %.2f'%(credit_df['Monthly_Inhand_Salary'].skew()))
plt.legend(loc = 'best')
plt.title('Customer Monthly Inhand Salary Distribution')

From the above graph, we can see that the degree of skewness is significantly reduced than compared to the skewness in the original data.

4.4.3 Log Transforming the column - Interest_Rate

In [ ]:
credit_df['Interest_Rate'].skew()

In [ ]:
### Understanding the distribution of the column - Interest_Rate

sns.distplot(credit_df['Interest_Rate'], label = 'Skewness: %.2f'%(credit_df['Interest_Rate'].skew()))
plt.legend(loc = 'best')
plt.title('Interest Rate Distribution')

In [ ]:
(credit_df['Interest_Rate'] < 0).any()

In [ ]:
(credit_df['Interest_Rate'] < 0).sum()

In [ ]:
### Understanding the distribution of the data log(Interest_Rate)

modified_interest = [np.log(interest) if interest > 0 else 0 for interest in credit_df['Interest_Rate']]
credit_df['Interest_Rate'] = modified_interest

sns.distplot(credit_df['Interest_Rate'], label = 'Skewness: %.2f'%(credit_df['Interest_Rate'].skew()))
plt.legend(loc = 'best')
plt.title('Interest Rate Distribution')

From the above graph, we can see that the degree of skewness is significantly reduced than compared to the skewness in the original data.

4.4.4 Log Transforming the column - Num_of_Delayed_Payment

In [ ]:
### Understanding the distribution of the column - Num_of_Delayed_Payment

sns.distplot(credit_df['Num_of_Delayed_Payment'], label = 'Skewness: %.2f'%(credit_df['Num_of_Delayed_Payment'].skew()))
plt.legend(loc = 'best')
plt.title('Delayed Payment Distribution')

In [ ]:
(credit_df['Num_of_Delayed_Payment']<0).any()

In [ ]:
(credit_df['Num_of_Delayed_Payment']<0).sum()

In [ ]:
### Understanding the distribution of the data log(Num_of_Delayed_Payment)

modified_payment = [np.log(payment) if payment > 0 else 0 for payment in credit_df['Num_of_Delayed_Payment']]
credit_df['Num_of_Delayed_Payment'] = modified_payment

sns.distplot(credit_df['Num_of_Delayed_Payment'], label = 'Skewness: %.2f'%(credit_df['Num_of_Delayed_Payment'].skew()))
plt.legend(loc = 'best')
plt.title('Delayed Payment Distribution')

From the above graph, we can see that the degree of skewness is significantly reduced than compared to the skewness in the original data.

4.4.5 Log Transforming the column - Num_Credit_Inquiries

In [ ]:
### Understanding the distribution of the column - Num_Credit_Inquiries

sns.distplot(credit_df['Num_Credit_Inquiries'], label = 'Skewness: %.2f'%(credit_df['Num_Credit_Inquiries'].skew()))
plt.legend(loc = 'best')
plt.title('Number of Credit Inquiries')

In [ ]:
(credit_df['Num_Credit_Inquiries']<0).any()

In [ ]:
(credit_df['Num_Credit_Inquiries']<0).sum()

In [ ]:
(credit_df['Num_Credit_Inquiries']).skew()

In [ ]:
### Understanding the distribution of the data log(Num_Credit_Inquiries)

modified_inquiries = [np.log(inquiries) if inquiries > 0 else 0 for inquiries in credit_df['Num_Credit_Inquiries']]
credit_df['Num_Credit_Inquiries'] = modified_inquiries

sns.distplot(credit_df['Num_Credit_Inquiries'], label = 'Skewness: %.2f'%(credit_df['Num_Credit_Inquiries'].skew()))
plt.legend(loc = 'best')
plt.title('Number of Credit Card Inquiries')

From the above graph, we can see that the degree of skewness is significantly reduced than compared to the skewness in the original data

4.4.6 Log Transforming the column - Total_EMI_per_month

In [ ]:
### Understanding the distribution of the column - Total_EMI_per_month

sns.distplot(credit_df['Total_EMI_per_month'], label = 'Skewness: %.2f'%(credit_df['Total_EMI_per_month'].skew()))
plt.legend(loc = 'best')
plt.title('Total EMI per month Distribution')

In [ ]:
(credit_df['Total_EMI_per_month']<0).any()

In [ ]:
(credit_df['Total_EMI_per_month']<0).sum()

In [ ]:
### Understanding the distribution of the data log(Total_EMI_per_month)

modified_emi = [np.log(emi) if emi > 0 else 0 for emi in credit_df['Total_EMI_per_month']]
credit_df['Total_EMI_per_month'] = modified_emi

sns.distplot(credit_df['Total_EMI_per_month'], label = 'Skewness: %.2f'%(credit_df['Total_EMI_per_month'].skew()))
plt.legend(loc = 'best')
plt.title('Total EMI per month Distribution')

From the above graph, we can see that the degree of skewness is significantly reduced than compared to the skewness in the original data.

4.4.7 Log Transforming the column - Amount_invested_monthly

In [ ]:
### Understanding the distribution of the column - Amount_invested_monthly

sns.distplot(credit_df['Amount_invested_monthly'], label = 'Skewness: %.2f'%(credit_df['Amount_invested_monthly'].skew()))
plt.legend(loc = 'best')
plt.title('Amount invested monthly Distribution')

In [ ]:
(credit_df['Amount_invested_monthly']<0).any()

In [ ]:
(credit_df['Amount_invested_monthly']<0).sum()

In [ ]:
### Understanding the distribution of the data log(Total_EMI_per_month)

modified_emi = [np.log(emi) if emi > 0 else 0 for emi in credit_df['Amount_invested_monthly']]
credit_df['Amount_invested_monthly'] = modified_emi

sns.distplot(credit_df['Amount_invested_monthly'], label = 'Skewness: %.2f'%(credit_df['Amount_invested_monthly'].skew()))
plt.legend(loc = 'best')
plt.title('Amount Invested Monthly Distribution')

From the above graph, we can see that the degree of skewness is significantly reduced than compared to the skewness in the original data.

In [ ]:
credit_df

4.5 Feature Encoding

Feature encoding is the process of turning categorical data in a dataset into numerical data. It is essential that we perform feature encoding because most machine learning models can only interpret numerical data and not data in text form.

In [ ]:
### One Hot Encoding the columns - Month, Occupation, Payment_of_Min_Amount of the dataset
# Here we are not doing for the credit score column because that is our target columns
encoded_dataset = pd.get_dummies(data = credit_df, 
                                       columns = ['Month', 'Occupation', 'Payment_of_Min_Amount'])
encoded_dataset

In [ ]:
### Encoding the Credit Score (Target) column
mapping = {'Poor': 0, 'Standard': 1, 'Good': 2}
encoded_dataset['Target'] = encoded_dataset['Credit_Score'].map(mapping)
encoded_dataset.drop(['Credit_Score'], axis=1, inplace=True) # Here we drop the Credit_Score col and make another column the target

In [ ]:
### Looking at the dataset

encoded_dataset

In [ ]:
encoded_dataset['Target'].value_counts()

In [ ]:
encoded_dataset.info()

In [ ]:
credit_df.info()

In [ ]:
# Set the display option to show all columns
pd.set_option('display.max_columns', None)

# Now, filter and display your DataFrame
Zero_encoded_dataset = encoded_dataset[encoded_dataset['Target'] == 0]  # Poor: 0

# This will now print all the columns without skipping any
Zero_encoded_dataset

In [ ]:
# Set the display option to show all columns
pd.set_option('display.max_columns', None)

# Now, filter and display your DataFrame
One_encoded_dataset = encoded_dataset[encoded_dataset['Target'] == 1] # Standard: 1

# This will now print all the columns without skipping any
One_encoded_dataset

In [ ]:
# Set the display option to show all columns
pd.set_option('display.max_columns', None)

# Now, filter and display your DataFrame
Two_encoded_dataset = encoded_dataset[encoded_dataset['Target'] == 2] # Good: 2

# This will now print all the columns without skipping any
Two_encoded_dataset

Now our datasets are ready for modelling.

5. Modelling

Scikit-learn is one of the most popular libraries for machine learning in Python and that is what we will use in the modelling part of this project.

Since Credit Score detection is a classfication problem, we will need to use classfication models, also known as classifiers, to train on our model to make predictions. I highly recommend checking out the scikit-learn documentation for more information on the different machine learning models available in their library. I have chosen the following classifiers for the job:

1. Logistic regression
2. Support vector classification
3. K-nearest neighbours
4. Naive Bayes
5. Decision tree
6. Random forest
   
Here, we will only be focusing on the data belonging to Good and Poor credit scores.

In this section of the notebook, I will fit the models to the training set as outlined above and evaluate their accuracy at making predictions. Once the best model is determined, I will also do hyperparameter tuning to further boost the performance of the best model.

`.values` → converts the DataFrame slice into a NumPy array (machine learning models usually need NumPy arrays instead of DataFrames).

In [ ]:
### Splitting the data to the matrices X and Y using the training set.
X = encoded_dataset.iloc[:, : -1].values # It means here we are taking all the rows and column except the last column(Target)
Y = encoded_dataset.iloc[:, -1].values # Here we are taking all the rows and only taking the last column(Target)

In [ ]:
### Looking at the new training data - X

X

In [ ]:
### Looking at the new test data - Y

Y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
### Dividing the dataset into train and test in the ratio of 70 : 30

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 27) #, shuffle = True


In [ ]:
X_train

In [ ]:
X_test

In [ ]:
Y_train

In [ ]:
Y_test

In [ ]:
from collections import OrderedDict

In [ ]:
### Dictionary to store model and its accuracy

model_accuracy = OrderedDict()

In [ ]:
### Dictionary to store model and its precision

model_precision = OrderedDict()

In [ ]:
### Dictionary to store model and its recall

model_recall = OrderedDict()

5.2.1 Applying Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
### Training the Logistic Regression model on the dataset

logistic_classifier = LogisticRegression(random_state = 27) # Here our model putting into the variable
logistic_classifier.fit(X_train, Y_train) # Fitting our model to our X_train & Y_train data

In [ ]:
### Predicting the Test set results

Y_pred = logistic_classifier.predict(X_test)

In [ ]:
# Here it's telling the what is the actual values and the prediction values
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [ ]:
### Making the confusion matrix

cm = confusion_matrix(Y_test, Y_pred)
print(cm)

### Printing the accuracy, precision, and recall of the model

logistic_accuracy = round(100 * accuracy_score(Y_test, Y_pred), 2)
model_accuracy['Logistic Regression'] = logistic_accuracy

logistic_precision = round(100 * precision_score(Y_test, Y_pred, average = 'weighted'), 2)
model_precision['Logistic Regression'] = logistic_precision

logistic_recall = round(100 * recall_score(Y_test, Y_pred, average = 'weighted'), 2)
model_recall['Logistic Regression'] = logistic_recall

print('The accuracy of this model is {} %.'.format(logistic_accuracy))
print('The precision of this model is {} %.'.format(logistic_precision))
print('The recall of this model is {} %.'.format(logistic_recall))

Class-by-class breakdown of `confusion_matrix`

Class 0 (row 0)

    Correctly predicted as class 0 → 3597
    Wrongly predicted as class 1 → 4867
    Wrongly predicted as class 2 → 200

Class 1 (row 1)

    Correctly predicted as class 1 → 12946
    Wrongly predicted as class 0 → 1898
    Wrongly predicted as class 2 → 1040

Class 2 (row 2)

    Correctly predicted as class 2 → 1273
    Wrongly predicted as class 0 → 78
    Wrongly predicted as class 1 → 3956

`confusion_matrix` → table showing correct vs. wrong predictions.

`accuracy` → overall score (how often you’re right).

`precision` → when you predict something, how often it’s correct.

`recall` → how well you find all actual positives.

5.2.2 Applying SVM 

In [ ]:
from sklearn.svm import SVC

In [ ]:
### Applying SVM Classification model (RBF kernel)
svm_classifier = SVC(kernel='rbf', random_state=27)
svm_classifier.fit(X_train, Y_train)

In [ ]:
### Predicting the Test set results
Y_pred = svm_classifier.predict(X_test)
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

In [ ]:
### Making the confusion matrix
cm = confusion_matrix(Y_test, Y_pred)
print(cm)

### Printing the accuracy, precision, and recall of the model
svm_accuracy = round(100 * accuracy_score(Y_test, Y_pred), 2)
model_accuracy['SVM (RBF kernel)'] = svm_accuracy

svm_precision = round(100 * precision_score(Y_test, Y_pred, average='weighted'), 2)
model_precision['SVM (RBF kernel)'] = svm_precision

svm_recall = round(100 * recall_score(Y_test, Y_pred, average='weighted'), 2)
model_recall['SVM (RBF kernel)'] = svm_recall

print('The accuracy of this model is {} %.'.format(svm_accuracy))
print('The precision of this model is {} %.'.format(svm_precision))
print('The recall of this model is {} %.'.format(svm_recall))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

We know that forKNN algo we need optimal k value so,For that below this code is writing for the optimal k value

In [ ]:
error_rates = []

# Calculate error rates for k = 1 to 30
for k in range(1, 31):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, Y_train)
    Y_pred = knn.predict(X_test)
    error_rates.append(1 - accuracy_score(Y_test, Y_pred))

# Find optimal k
optimal_k = range(1, 31)[error_rates.index(min(error_rates))]
#print("Optimal K is:", optimal_k)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(range(1, 31), error_rates, color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=8)

# Draw red vertical line at optimal k
plt.axvline(x=optimal_k, color='red', linestyle='--', linewidth=2, label=f'Optimal k = {optimal_k}')

plt.xlabel('K Value')
plt.ylabel('Error Rate')
plt.title('Error Rate vs K Value')
plt.legend()
plt.show()


Here we got our optimal k value which is `4`.

Now we will implement on our model so before directly putting the 4 we will also explore that this is correct or not and we will put the surrounding values of this like: 1,3 and 5.

5.2.3 Applying K-Nearest Neighbors (k = 1)

In [ ]:
### Applying 1NN model

classifier_1nn = KNeighborsClassifier(n_neighbors = 1, algorithm = 'auto', p = 2, metric = 'minkowski')
classifier_1nn.fit(X_train, Y_train)

In [ ]:
### Predicting the Test set results

Y_pred = classifier_1nn.predict(X_test)
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

In [ ]:
### Making the confusion matrix

cm = confusion_matrix(Y_test, Y_pred)
print(cm)

### Printing the accuracy, precision, and recall of the model

nn1_accuracy = round(100 * accuracy_score(Y_test, Y_pred), 2)
model_accuracy['1 - Nearest Neighbors'] = nn1_accuracy

nn1_precision = round(100 * precision_score(Y_test, Y_pred, average = 'weighted'), 2)
model_precision['1 - Nearest Neighbors'] = nn1_precision

nn1_recall = round(100 * recall_score(Y_test, Y_pred, average = 'weighted'), 2)
model_recall['1 - Nearest Neighbors'] = nn1_recall

print('The accuracy of this model is {} %.'.format(nn1_accuracy))
print('The precision of this model is {} %.'.format(nn1_precision))
print('The recall of this model is {} %.'.format(nn1_recall))

5.2.4 Applying K - Nearest Neighbors (k = 3)

In [ ]:
### Applying 3NN model

classifier_3nn = KNeighborsClassifier(n_neighbors = 3, algorithm = 'auto', p = 2, metric = 'minkowski')
classifier_3nn.fit(X_train, Y_train)

In [ ]:
### Predicting the Test set results

Y_pred = classifier_3nn.predict(X_test)
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

In [ ]:
### Making the confusion matrix

cm = confusion_matrix(Y_test, Y_pred)
print(cm)

### Printing the accuracy, precision, and recall of the model

nn3_accuracy = round(100 * accuracy_score(Y_test, Y_pred), 2)
model_accuracy['3 - Nearest Neighbors'] = nn3_accuracy

nn3_precision = round(100 * precision_score(Y_test, Y_pred, average = 'weighted'), 2)
model_precision['3 - Nearest Neighbors'] = nn3_precision

nn3_recall = round(100 * recall_score(Y_test, Y_pred, average = 'weighted'), 2)
model_recall['3 - Nearest Neighbors'] = nn3_recall

print('The accuracy of this model is {} %.'.format(nn3_accuracy))
print('The precision of this model is {} %.'.format(nn3_precision))
print('The recall of this model is {} %.'.format(nn3_recall))

5.2.5 Applying K - Nearest Neighbors (k = 4)

In [ ]:
### Applying 4NN model

classifier_4nn = KNeighborsClassifier(n_neighbors = 4, algorithm = 'auto', p = 2, metric = 'minkowski')
classifier_4nn.fit(X_train, Y_train)

In [ ]:
### Predicting the Test set results

Y_pred = classifier_4nn.predict(X_test)
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

In [ ]:
### Making the confusion matrix

cm = confusion_matrix(Y_test, Y_pred)
print(cm)

### Printing the accuracy, precision, and recall of the model

nn4_accuracy = round(100 * accuracy_score(Y_test, Y_pred), 2)
model_accuracy['4 - Nearest Neighbors'] = nn4_accuracy

nn4_precision = round(100 * precision_score(Y_test, Y_pred, average = 'weighted'), 2)
model_precision['4 - Nearest Neighbors'] = nn4_precision

nn4_recall = round(100 * recall_score(Y_test, Y_pred, average = 'weighted'), 2)
model_recall['4 - Nearest Neighbors'] = nn4_recall

print('The accuracy of this model is {} %.'.format(nn4_accuracy))
print('The precision of this model is {} %.'.format(nn4_precision))
print('The recall of this model is {} %.'.format(nn4_recall))

5.2.6 Applying K - Nearest Neighbors (k = 5)

In [ ]:
### Applying 5NN model

classifier_5nn = KNeighborsClassifier(n_neighbors = 5, algorithm = 'auto', p = 2, metric = 'minkowski')
classifier_5nn.fit(X_train, Y_train)

In [ ]:
### Predicting the Test set results

Y_pred = classifier_5nn.predict(X_test)
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

In [ ]:
### Making the confusion matrix

cm = confusion_matrix(Y_test, Y_pred)
print(cm)

### Printing the accuracy, precision, and recall of the model

nn5_accuracy = round(100 * accuracy_score(Y_test, Y_pred), 2)
model_accuracy['5 - Nearest Neighbors'] = nn5_accuracy

nn5_precision = round(100 * precision_score(Y_test, Y_pred, average = 'weighted'), 2)
model_precision['5 - Nearest Neighbors'] = nn5_precision

nn5_recall = round(100 * recall_score(Y_test, Y_pred, average = 'weighted'), 2)
model_recall['5 - Nearest Neighbors'] = nn5_recall

print('The accuracy of this model is {} %.'.format(nn5_accuracy))
print('The precision of this model is {} %.'.format(nn5_precision))
print('The recall of this model is {} %.'.format(nn5_recall))

In [ ]:
### Looking at the accuracy graph of all the nearest neighbors

labels = ['1NN', '3NN', '4NN', '5NN'] #,'optimal_knn'
values = [nn1_accuracy, nn3_accuracy, nn4_accuracy, nn5_accuracy] #,nnoptimal_k_accuracy

plt.title('Accuracies of all the nearest neighbor models')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.plot(labels, values, '*-')
plt.show()

Now let's plot the accuracies of all the nearest neighbor models. We can see that the accuracy first increases, reaches a peak and then decreases.

For knn algo for k=4 we got our best accuracy score also we can see this diagram also.

5.2.7 Applying Gaussian Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
### Applying Naive Bayes Classification model

naive_bayes_classifier = GaussianNB()
naive_bayes_classifier.fit(X_train, Y_train)

In [ ]:
### Predicting the Test set results

Y_pred = naive_bayes_classifier.predict(X_test)
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

In [ ]:
### Making the confusion matrix

cm = confusion_matrix(Y_test, Y_pred)
print(cm)

### Printing the accuracy, precision, and recall of the model

naive_bayes_accuracy = round(100 * accuracy_score(Y_test, Y_pred), 2)
model_accuracy['Gaussian Naive Bayes'] = naive_bayes_accuracy

naive_bayes_precision = round(100 * precision_score(Y_test, Y_pred, average = 'weighted'), 2)
model_precision['Gaussian Naive Bayes'] = naive_bayes_precision

naive_bayes_recall = round(100 * recall_score(Y_test, Y_pred, average = 'weighted'), 2)
model_recall['Gaussian Naive Bayes'] = naive_bayes_recall

print('The accuracy of this model is {} %.'.format(naive_bayes_accuracy))
print('The precision of this model is {} %.'.format(naive_bayes_precision))
print('The recall of this model is {} %.'.format(naive_bayes_recall))

5.2.8 Applying Decision Tree Classification

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
### Applying Decision Tree Classification model

decision_tree_classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 27) # Here we are using the methode of entropy
decision_tree_classifier.fit(X_train, Y_train)

In [ ]:
### Predicting the Test set results

Y_pred = decision_tree_classifier.predict(X_test)
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

In [ ]:
### Making the confusion matrix

cm = confusion_matrix(Y_test, Y_pred)
print(cm)

### Printing the accuracy, precision, and recall of the model

decision_tree_accuracy = round(100 * accuracy_score(Y_test, Y_pred), 2)
model_accuracy['Decision Tree'] = decision_tree_accuracy

decision_tree_precision = round(100 * precision_score(Y_test, Y_pred, average = 'weighted'), 2)
model_precision['Decision Tree'] = decision_tree_precision

decision_tree_recall = round(100 * recall_score(Y_test, Y_pred, average = 'weighted'), 2)
model_recall['Decision Tree'] = decision_tree_recall

print('The accuracy of this model is {} %.'.format(decision_tree_accuracy))
print('The precision of this model is {} %.'.format(decision_tree_precision))
print('The recall of this model is {} %.'.format(decision_tree_recall))

5.2.9 Applying Random Forest Classification (10 trees)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# To store error rates
error_rates = []

# Loop through possible n_estimators values
for n in range(10, 201, 10):  # from 10 to 200 with step 10
    rf = RandomForestClassifier(n_estimators=n, random_state=42)
    rf.fit(X_train, Y_train)
    Y_pred = rf.predict(X_test)
    error_rates.append(1 - accuracy_score(Y_test, Y_pred))

# Find optimal n_estimators
n_values = list(range(10, 201, 10))
optimal_n = n_values[error_rates.index(min(error_rates))]
#print("Optimal n_estimators is:", optimal_n)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(n_values, error_rates, color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=8)

# Add vertical red line at optimal point
plt.axvline(x=optimal_n, color='red', linestyle='--', label=f'Optimal n_estimators = {optimal_n}')

plt.xlabel('Number of Trees (n_estimators)')
plt.ylabel('Error Rate')
plt.title('Error Rate vs n_estimators in Random Forest')
plt.legend()
plt.show()

In [ ]:
### Applying Random Forest Classification model (10 trees)

random_forest_10_classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 27)
random_forest_10_classifier.fit(X_train, Y_train)

In [ ]:
### Predicting the Test set results

Y_pred = random_forest_10_classifier.predict(X_test)
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

In [ ]:
### Making the confusion matrix

cm = confusion_matrix(Y_test, Y_pred)
print(cm)

### Printing the accuracy, precision, and recall of the model

random_forest_10_accuracy = round(100 * accuracy_score(Y_test, Y_pred), 2)
model_accuracy['Random Forest (10 trees)'] = random_forest_10_accuracy

random_forest_10_precision = round(100 * precision_score(Y_test, Y_pred, average = 'weighted'), 2)
model_precision['Random Forest (10 trees)'] = random_forest_10_precision

random_forest_10_recall = round(100 * recall_score(Y_test, Y_pred, average = 'weighted'), 2)
model_recall['Random Forest (10 trees)'] = random_forest_10_recall

print('The accuracy of this model is {} %.'.format(random_forest_10_accuracy))
print('The precision of this model is {} %.'.format(random_forest_10_precision))
print('The recall of this model is {} %.'.format(random_forest_10_recall))

5.2.10 Applying Random Forest Classification (50 trees)

In [ ]:
### Applying Random Forest Classification model (50 trees)

random_forest_50_classifier = RandomForestClassifier(n_estimators = 50, criterion = 'entropy', random_state = 27)
random_forest_50_classifier.fit(X_train, Y_train)

In [ ]:
### Predicting the Test set results

Y_pred = random_forest_50_classifier.predict(X_test)
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

In [ ]:
### Making the confusion matrix

cm = confusion_matrix(Y_test, Y_pred)
print(cm)

### Printing the accuracy, precision, and recall of the model

random_forest_50_accuracy = round(100 * accuracy_score(Y_test, Y_pred), 2)
model_accuracy['Random Forest (50 trees)'] = random_forest_50_accuracy

random_forest_50_precision = round(100 * precision_score(Y_test, Y_pred, average = 'weighted'), 2)
model_precision['Random Forest (50 trees)'] = random_forest_50_precision

random_forest_50_recall = round(100 * recall_score(Y_test, Y_pred, average = 'weighted'), 2)
model_recall['Random Forest (50 trees)'] = random_forest_50_recall

print('The accuracy of this model is {} %.'.format(random_forest_50_accuracy))
print('The precision of this model is {} %.'.format(random_forest_50_precision))
print('The recall of this model is {} %.'.format(random_forest_50_recall))

5.2.11 Applying Random Forest Classification (100 trees)

In [ ]:
### Applying Random Forest Classification model (100 trees)

random_forest_100_classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 27)
random_forest_100_classifier.fit(X_train, Y_train)

In [ ]:
### Predicting the Test set results

Y_pred = random_forest_100_classifier.predict(X_test)
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

In [ ]:
### Making the confusion matrix

cm = confusion_matrix(Y_test, Y_pred)
print(cm)

### Printing the accuracy, precision, and recall of the model

random_forest_100_accuracy = round(100 * accuracy_score(Y_test, Y_pred), 2)
model_accuracy['Random Forest (100 trees)'] = random_forest_100_accuracy

random_forest_100_precision = round(100 * precision_score(Y_test, Y_pred, average = 'weighted'), 2)
model_precision['Random Forest (100 trees)'] = random_forest_100_precision

random_forest_100_recall = round(100 * recall_score(Y_test, Y_pred, average = 'weighted'), 2)
model_recall['Random Forest (100 trees)'] = random_forest_100_recall

print('The accuracy of this model is {} %.'.format(random_forest_100_accuracy))
print('The precision of this model is {} %.'.format(random_forest_100_precision))
print('The recall of this model is {} %.'.format(random_forest_100_recall))

5.2.12 Applying Random Forest Classification (190 trees)

In [ ]:
### Applying Random Forest Classification model (190 trees)

random_forest_190_classifier = RandomForestClassifier(n_estimators = 190, criterion = 'entropy', random_state = 27)
random_forest_190_classifier.fit(X_train, Y_train)

In [ ]:
### Predicting the Test set results

Y_pred = random_forest_190_classifier.predict(X_test)
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

In [ ]:
### Making the confusion matrix

cm = confusion_matrix(Y_test, Y_pred)
print(cm)

### Printing the accuracy, precision, and recall of the model

random_forest_190_accuracy = round(100 * accuracy_score(Y_test, Y_pred), 2)
model_accuracy['Random Forest (190 trees)'] = random_forest_190_accuracy

random_forest_190_precision = round(100 * precision_score(Y_test, Y_pred, average = 'weighted'), 2)
model_precision['Random Forest (190 trees)'] = random_forest_190_precision

random_forest_190_recall = round(100 * recall_score(Y_test, Y_pred, average = 'weighted'), 2)
model_recall['Random Forest (190 trees)'] = random_forest_190_recall

print('The accuracy of this model is {} %.'.format(random_forest_190_accuracy))
print('The precision of this model is {} %.'.format(random_forest_190_precision))
print('The recall of this model is {} %.'.format(random_forest_190_recall))

5.2.13 Applying Random Forest Classification (200 trees)

In [ ]:
### Applying Random Forest Classification model (200 trees)

random_forest_200_classifier = RandomForestClassifier(n_estimators = 200, criterion = 'entropy', random_state = 27)
random_forest_200_classifier.fit(X_train, Y_train)

In [ ]:
### Predicting the Test set results

Y_pred = random_forest_200_classifier.predict(X_test)
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

In [ ]:
### Making the confusion matrix

cm = confusion_matrix(Y_test, Y_pred)
print(cm)

### Printing the accuracy, precision, and recall of the model

random_forest_200_accuracy = round(100 * accuracy_score(Y_test, Y_pred), 2)
model_accuracy['Random Forest (200 trees)'] = random_forest_200_accuracy

random_forest_200_precision = round(100 * precision_score(Y_test, Y_pred, average = 'weighted'), 2)
model_precision['Random Forest (200 trees)'] = random_forest_200_precision

random_forest_200_recall = round(100 * recall_score(Y_test, Y_pred, average = 'weighted'), 2)
model_recall['Random Forest (200 trees)'] = random_forest_200_recall

print('The accuracy of this model is {} %.'.format(random_forest_200_accuracy))
print('The precision of this model is {} %.'.format(random_forest_200_precision))
print('The recall of this model is {} %.'.format(random_forest_200_recall))

In [ ]:
### Looking at the accuracy graph of all the Random Forest

labels = ['10', '50', '100', '190', '200']
values = [
    random_forest_10_accuracy,
    random_forest_50_accuracy,
    random_forest_100_accuracy,
    random_forest_190_accuracy,
    random_forest_200_accuracy
]

plt.title('Accuracies of all the Random Forest models')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.plot(labels, values, '*-')
plt.show()


By doing above all those thing and exploring we get best n_estimator is `200` also we can see in the above diagram.

Since 190 and 200 give nearly identical accuracy (difference of just 0.03%), choosing 190 makes sense because:


* You reduce computation time (fewer trees to train and predict).


* You reduce noise — adding more trees beyond a certain point rarely changes accuracy much, but it can make the model slightly more sensitive to tiny fluctuations in the dataset.


* From a practical point of view, both are statistically the same — the tiny difference could just be due to the randomness in your train-test split.


So, picking 190 is a perfectly reasonable and efficient choice here.

5.2.13 Applying Stacking Classifier

In [ ]:
from sklearn.ensemble import StackingClassifier

In [ ]:
print('The accuracy of this model logistic_accuracy is {} %.'.format(logistic_accuracy))
print('The accuracy of this model nn4_accuracy is {} %.'.format(nn4_accuracy))
print('The accuracy of this model naive_bayes_accuracy is {} %.'.format(naive_bayes_accuracy))
print('The accuracy of this model decision_tree_accuracy is {} %.'.format(decision_tree_accuracy))
print('The accuracy of this model random_forest_190_accuracy is {} %.'.format(random_forest_190_accuracy))
print('The accuracy of this model svm_accuracy is {} %.'.format(svm_accuracy))

In [ ]:
### Preparing the Stacking Classifier

### Define the base models

base_models = list()

base_models.append(('4nn', classifier_4nn))
base_models.append(('decision_tree', decision_tree_classifier))
base_models.append(('random_forest_190', random_forest_190_classifier))

### Define the meta models

meta_model = logistic_classifier

In [ ]:
### Applying Stacking Classification

stacking_classifier = StackingClassifier(estimators = base_models, final_estimator = meta_model)
stacking_classifier.fit(X_train, Y_train)

In [ ]:
### Predicting the Test set results

Y_pred = stacking_classifier.predict(X_test)
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

In [ ]:
### Making the confusion matrix

cm = confusion_matrix(Y_test, Y_pred)
print(cm)

### Printing the accuracy, precision, and recall of the model

stacking_accuracy = round(100 * accuracy_score(Y_test, Y_pred), 2)
model_accuracy['Stacking Classification'] = stacking_accuracy

stacking_precision = round(100 * precision_score(Y_test, Y_pred, average = 'weighted'), 2)
model_precision['Stacking Classification'] = stacking_precision

stacking_recall = round(100 * recall_score(Y_test, Y_pred, average = 'weighted'), 2)
model_recall['Stacking Classification'] = stacking_recall

print('The accuracy of this model is {} %.'.format(stacking_accuracy))
print('The precision of this model is {} %.'.format(stacking_precision))
print('The recall of this model is {} %.'.format(stacking_recall))

In [ ]:
import pickle

In [ ]:
# # Assume 'stacking_classifier' is your already trained model object

# # 1. Define the filename for your model
# filename = 'stacking_model.pkl'

# # 2. Open the file in write-binary ('wb') mode and save the model
# with open(filename, 'wb') as file:
#     pickle.dump(stacking_classifier, file)

# print(f"Model saved successfully to '{filename}' ✅")

5.2.14 Applying Soft Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
### Preparing the Soft Voting Classifier

### Creating the list of estimators

estimators = list()

estimators.append(('4nn', classifier_4nn))
estimators.append(('decision_tree', decision_tree_classifier))
estimators.append(('random_forest_190', random_forest_190_classifier))

In [ ]:
### Applying Soft Voting Classification

soft_voting_classifier = VotingClassifier(estimators = estimators, voting = 'soft')
soft_voting_classifier.fit(X_train, Y_train)

In [ ]:
### Predicting the Test set results

Y_pred = soft_voting_classifier.predict(X_test)
print(np.concatenate((Y_pred.reshape(len(Y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

In [ ]:
### Making the confusion matrix

cm = confusion_matrix(Y_test, Y_pred)
print(cm)

### Printing the accuracy, precision, and recall of the model

soft_voting_accuracy = round(100 * accuracy_score(Y_test, Y_pred), 2)
model_accuracy['Soft Voting'] = soft_voting_accuracy

soft_voting_precision = round(100 * precision_score(Y_test, Y_pred, average = 'weighted'), 2)
model_precision['Soft Voting'] = soft_voting_precision

soft_voting_recall = round(100 * recall_score(Y_test, Y_pred, average = 'weighted'), 2)
model_recall['Soft Voting'] = soft_voting_recall

print('The accuracy of this model is {} %.'.format(soft_voting_accuracy))
print('The precision of this model is {} %.'.format(soft_voting_precision))
print('The recall of this model is {} %.'.format(soft_voting_recall))

5.3 Model evaluation

Model evaluation is the process of using different evaluation metrics to understand a machine learning model's performance, as well as its strengths and weaknesses.

5.3.1 Training accuracy of the models

Now we will tabulate all the models along with their accuracies. This data is stored in the model_performance dictionary. We will use the tabulate package for tabulating the results.

In [ ]:
### Looking at the model accuracy dictionary

model_accuracy

In [ ]:
### Looking at the model precision dictionary

model_precision

In [ ]:
### Looking at the model recall dictionary

model_recall

In [ ]:
from tabulate import tabulate

In [ ]:
### Tabulating the results

table = []
table.append(['S.No.', 'Classification Model', 'Model Accuracy', 'Model Precision', 'Model Recall'])
count = 1

for model in model_accuracy:
    row = [count, model, model_accuracy[model], model_precision[model], model_recall[model]]
    table.append(row)
    count += 1
    
print(tabulate(table, headers = 'firstrow', tablefmt = 'fancy_grid'))

From the above table, we can see that Stacking Classifier has the highest model accuracy of 81.53% among all the other models. It also has the highest precision and recall with 81.5% and 81.53%.

6. Conclusion

Hence our recommended model - Stacking Classifier provides a model accuracy of 81.53 percent, a model precision of 81.5 percent, and a model recall of 81.53 percent.